In [1]:
import os
import sys
PATH = os.getcwd()
DIR_DATA = PATH + '{0}data{0}'.format(os.sep)
sys.path.append(PATH) if PATH not in list(sys.path) else None

In [7]:
import json
import random
import spacy
from spacy.training import Example
from spacy.training import offsets_to_biluo_tags

In [8]:
# Cargar el modelo base (puede ser "es_core_news_sm" o "blank")
nlp = spacy.blank("es")  # modelo vacío en español

In [9]:
# Crear el componente NER y añadirlo al pipeline
ner = nlp.add_pipe("ner")

In [10]:
text = "Apple lanzó el nuevo iPhone en California."
entities = [(0, 5, "ORG"), (32, 42, "LOC")]

print(offsets_to_biluo_tags(nlp.make_doc(text), entities))

['U-ORG', 'O', 'O', 'O', 'O', 'O', '-', '-']


In [11]:
# Cargar los datos anotados desde el JSON
with open(DIR_DATA + "datasetNER.json", "r", encoding="utf8") as f:
    TRAIN_DATA = json.load(f)

In [12]:
# Añadir las etiquetas al componente NER
for entry in TRAIN_DATA:
    for _, _, label in entry["entities"]:
        ner.add_label(label)

In [13]:
# Entrenamiento supervisado
optimizer = nlp.initialize()

In [ ]:
for epoch in range(30):
    random.shuffle(TRAIN_DATA)
    losses = {}
    for entry in TRAIN_DATA:
        doc = nlp.make_doc(entry["text"])
        example = Example.from_dict(doc, {"entities": entry["entities"]})
        nlp.update([example], sgd=optimizer, losses=losses)
    print(f"Epoch {epoch+1} - Pérdida: {losses['ner']:.4f}")

C:\Users\epuerta\AppData\Local\anaconda3\Lib\site-packages\spacy\training\iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Apple lanzó el nuevo iPhone en California." with entities "[[0, 5, 'ORG'], [32, 42, 'LOC']]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(


In [ ]:
# Guardar el modelo entrenado
nlp.to_disk("modelo_ner_personalizado")
print("Modelo guardado en 'modelo_ner_personalizado'")

In [ ]:
nlp2 = spacy.load("modelo_ner_personalizado")
texto = "Google abrió una nueva oficina en Madrid."
doc = nlp2(texto)
for ent in doc.ents:
    print(ent.text, ent.label_)